In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr
from data_generator import DataGenerator, batch_predict

2025-07-15 20:54:53.891677: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-15 20:54:53.927096: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-15 20:54:53.927119: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-15 20:54:53.927948: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-15 20:54:53.933584: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# I think we can get the metrics in a for loop for all the blocks individually?
# And later we will compute an average metric too

In [3]:
# get teh trained model?
overlapping_model = tf.keras.models.load_model("models/CNN_seq2seq_overlapping.keras")

2025-07-15 20:56:11.070436: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31134 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:06:00.0, compute capability: 7.0


In [4]:
# locate the test data

# input features
input_features_loc = 'data/Overlapping_data/Test_input_features'
input_contents = os.listdir(input_features_loc)
input_contents.sort()

In [5]:
input_contents

['test_data_blk_0103.npy',
 'test_data_blk_0104.npy',
 'test_data_blk_0105.npy',
 'test_data_blk_0106.npy',
 'test_data_blk_0201.npy',
 'test_data_blk_0202.npy',
 'test_data_blk_0205.npy',
 'test_data_blk_0206.npy',
 'test_data_blk_0302.npy',
 'test_data_blk_0303.npy',
 'test_data_blk_0304.npy',
 'test_data_blk_0305.npy',
 'test_data_blk_0306.npy']

In [6]:
# test targets
out_targets_loc = 'data/Overlapping_data/Test_out_targets'
out_contents = os.listdir(out_targets_loc)
out_contents.sort()

In [7]:
out_contents

['test_targets_blk_0103.npy',
 'test_targets_blk_0104.npy',
 'test_targets_blk_0105.npy',
 'test_targets_blk_0106.npy',
 'test_targets_blk_0201.npy',
 'test_targets_blk_0202.npy',
 'test_targets_blk_0205.npy',
 'test_targets_blk_0206.npy',
 'test_targets_blk_0302.npy',
 'test_targets_blk_0303.npy',
 'test_targets_blk_0304.npy',
 'test_targets_blk_0305.npy',
 'test_targets_blk_0306.npy']

In [8]:
trial_test_features = np.load(os.path.join(input_features_loc, input_contents[0]))

In [9]:
trial_test_features.shape

(12288, 13, 32, 32, 3)

In [10]:
trial_test_targets = np.load(os.path.join(out_targets_loc, out_contents[0]))

In [11]:
trial_test_targets.shape

(12288, 7, 32)

In [12]:
batch_size = 32
test_data_gen = DataGenerator(trial_test_features, trial_test_targets, batch_size, shuffle=False)

In [13]:
%%time
test_preds, test_targets = batch_predict(overlapping_model, test_data_gen, flatten=True, verbose=False)

2025-07-15 20:56:17.337669: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907


Predictions: 12288
True: 12288
CPU times: user 23.5 s, sys: 1.33 s, total: 24.8 s
Wall time: 22.8 s


In [14]:
test_preds.shape

(12288, 7, 32)

In [15]:
test_targets.shape

(12288, 7, 32)

In [16]:
np.mean(trial_test_targets == test_targets)

1.0

In [ ]:
%%time
# Get preds in a loop
# now how to proceed? We may need to do a batch predict now using the generator 

test_rmse = []
test_mae = []
test_r2 = []
test_personr = []
preds = []
for i in range(len(input_contents)):
    # load the features
    test_features = np.load(os.path.join(input_features_loc, input_contents[i]))
    # load targets
    test_targets = np.load(os.path.join(out_targets_loc, out_contents[i]))
    test_data_gen = DataGenerator(test_features, test_targets, batch_size, shuffle=False)
    # Use the batch predictions to generate the predictions
    test_preds, test_targets_alt = batch_predict(overlapping_model, test_data_gen, flatten=True, verbose=True)
    preds.append(test_preds)
    print(np.mean(test_targets == test_targets_alt))
    # compute the test scores, I think we need to flatten these before computing the scores - or can use tf, but the answers are going to be the same
    test_preds_flatten = test_preds.flatten()
    test_targets_flatten = test_targets_alt.flatten()
    mae = mean_absolute_error(test_targets_flatten, test_preds_flatten)
    test_mae.append(mae)
    rmse = np.sqrt(mean_squared_error(test_targets_flatten, test_preds_flatten))
    test_rmse.append(rmse)
    rsquare = r2_score(test_targets_flatten, test_preds_flatten)
    test_r2.append(rsquare)
    pearsonr_score = pearsonr(test_targets_flatten, test_preds_flatten)[0]
    test_personr.append(pearsonr_score)

In [18]:
test_rmse

[0.31167305,
 0.29188478,
 0.30059206,
 0.28810465,
 0.31843045,
 0.30274057,
 0.29404983,
 0.3454868,
 0.3126328,
 0.30507272,
 0.30527887,
 0.33070317,
 0.3213102]

In [19]:
test_mae

[0.14829092,
 0.1346112,
 0.1401546,
 0.1311867,
 0.15043406,
 0.14265612,
 0.13445367,
 0.1619271,
 0.1468941,
 0.14274958,
 0.1413797,
 0.15627992,
 0.14830215]

In [20]:
test_r2

[0.7556340761967968,
 0.8107784070331502,
 0.7912395480712032,
 0.8101154122873547,
 0.7476560114900077,
 0.7426032448909583,
 0.8108243443619754,
 0.7055944086275721,
 0.7548352290157057,
 0.7501988662353444,
 0.7723581877459491,
 0.7297923720287193,
 0.7505136420795282]

In [21]:
test_pearsonr

NameError: name 'test_pearsonr' is not defined

In [20]:
# Now what about all test data taken together?

In [21]:
# The kernel keeps restarting, so we have to take whatever we have here below this cell to a .py script

In [22]:
all_test_features_list = [np.load(os.path.join(input_features_loc, file)) for file in input_contents]

In [23]:
all_test_features_list[0].shape

(12288, 13, 32, 32, 3)

In [24]:
len(all_test_features_list)

13

In [ ]:
# stack all data together
all_test_features = np.vstack(all_test_features_list)

In [ ]:
all_test_features.shape

In [ ]:
all_test_targets_list = [np.load(os.path.join(out_targets_loc, file)) for file in out_contents]

In [ ]:
all_test_targets_list[0].shape

In [ ]:
len(all_test_targets_list)

In [ ]:
# stack all data together
all_test_targets = np.vstack(all_test_targets_list)

In [ ]:
all_test_targets.shape

In [ ]:
# create the test data generator
all_test_data_gen = (all_test_features, all_test_targets, batch_size, shuffle = False)

In [ ]:
# get all test predictions
all_test_preds, all_test_targets_alt = batch_predict(overlapping_model, all_test_data_gen, flatten=True, verbose=False)

In [ ]:
all_test_preds.shape, all_test_targets_alt.shape

In [ ]:
np.mean(all_test_targets_alt == all_test_targets)

In [ ]:
# flatten these vectors
all_test_targets_flatten = all_test_targets_alt.flatten()
all_test_preds_flatten = all_test_preds.flatten()

In [ ]:
all_test_targets_flatten.shape, all_test_preds_flatten.shape

In [ ]:
# compute metrics
all_test_rmse = np.sqrt(mean_squared_error(all_test_targets_flatten, all_test_preds_flatten))
all_test_mae = mean_absolute_error(all_test_targets_flatten, all_test_preds_flatten)
all_test_r2_score = r2_score(all_test_targets_flatten, all_test_preds_flatten)
all_test_pearsonr = pearsonr(all_test_targets_flatten, all_test_preds_flatten)

In [ ]:
all_test_rmse, all_test_mae, all_test_r2_score, all_test_pearsonr[0]